# OCR Matching

In this notebook we are going to build a tool that taken as input a medical prescription in pdf,jpg format or a string gives me as output the list of facilities that perform medical analysis.

To extract information from prescriptions we will use OCR technology while to match the words in the prescription with those in the dataset we will use the fuzzywuzzy library.

OCR pdf: https://pypi.org/project/PyPDF2/

OCR jpg: https://www.jaided.ai/easyocr/tutorial/

FuzzyWuzzy: https://pypi.org/project/fuzzywuzzy/

In [1]:
pip install  pypdf langchain tika typing fuzzywuzzy pymupdf easyocr

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Simone\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from tika import parser
from typing import List
import re
from typing import List, Tuple
from fuzzywuzzy import fuzz, process
import fitz
import easyocr
import numpy as np


c:\Users\Simone\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
dataset = pd.read_csv('https://github.com/SimoneFarallo/public_and_social_services/raw/main/data/final_data_cleaned.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset.head()

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,POLIAMBULATORIALE PARABIAGO,PARABIAGO,MI,VIA XI FEBBRAIO N. 31,ATS DELLA CITTA' METROPOLITANA DI MILANO,2340.0,Struttura ambulatoriale,0331449633,Info non disponibile,No,14.56
1,ALTRE TRAZIONI CUTANEE DEGLI ARTI,9346,ORTOPEDIA E TRAUMATOLOGIA,ATT. CLINICA,POLIAMBULATORIO OSP. MORBEGNO,MORBEGNO,SO,VIA MORELLI 1,ATS DELLA MONTAGNA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,11.36
2,STREPTOCOCCO ANTICORPI ANTI ANTISTREPTOLISINA-...,91085,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,POLIAMB. A.O.IST.OSPIT.CREMONA,CREMONA,CR,VIA LARGO PRIORI N.1,ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,4.85
3,MONITORAGGIO INCRUENTO DELLA SATURAZIONE ARTER...,89655,PNEUMOLOGIA,ATT. CLINICA,POLIAMB. IRCCS MAUGERI,PAVIA,PV,"VIA MAUGERI, 10",ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,9.3
4,TEST ANTIGENICI (RAPID DIAGNOSTICS TEST Ag-RDT...,90557,MICROBIOLOGIA-VIROLOGIA,LABORATORIO,OSPEDALE BASSINI - CINISELLO BALSAMO,CINISELLO BALSAMO,MI,VIA GORKI 50,ATS DELLA CITTA' METROPOLITANA DI MILANO,30058.0,Struttura di ricovero e cura,02618311,026127543,No,Info non disponibile


## OCR Matching for pdf

I create a function that taken as input a pdf, gives me as output a list

In [4]:
# Function to match  prescriptions with a dataset 
def match_recipe_to_dataset_pdf(pdf_file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:

    # Check if pdf_file is a string, if not raise TypeError
    if not isinstance(pdf_file, str):
        raise TypeError("The PDF file path must be a string.")
    
    # Check if dataset is a pandas dataframe, if not raise TypeError
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("The dataset must be a pandas.DataFrame object.")

    # Extract text from pdf file using fitz library
    pdf_file = fitz.open(pdf_file)
    text = ''
    for page in pdf_file:
        text += page.get_text()

    # Find words of interest using regex 
    words = re.findall(r' - ([A-Za-z\s]+)', text) # Initial regex function to extract words from recipe (starting with ' - ([A-Z]+))'

    # Match words with words in dataset 'Codice prestazione ambulatoriale' column using fuzzywuzzy
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("The word of interest must be a string.") # Function to raise error for non-string input
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1) # Use fuzzywuzzy library to score the similarity between two strings 
        filtered_scores = [score for score in scores if score[1] >= threshold] # Filter scores based on the threshold value
        for score in filtered_scores:
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows() # Select the output rows based on the matching score
            matches.extend([(word, row[1], score[1]) for row in rows]) # Add the matched words, output rows, and score to the list 'matches'

    # Sort the results based on score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)
    
    # Return a list of tuples containing matched word, output row, and score
    return matches_sorted

In [5]:
cd C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


C:\Users\Simone\Documents\Desktop\public_and_social_services\ricette


In [6]:
match_recipe_to_dataset_pdf('ricetta_1.pdf',dataset)

[('DIIDROTESTOSTERONE DHT\n',
  Codice prestazione ambulatoriale    DIIDROTESTOSTERONE (DHT)
  Codice branca regionale                                90175
  Branca regionale                             CHIMICA CLINICA
  Tipo di prestazione                              LABORATORIO
  Nome struttura                         POLICLINICO SAN MARCO
  Comune struttura                                  OSIO SOTTO
  Provincia struttura                                       BG
  Indirizzo                                  CORSO EUROPA N. 7
  Descrizione ASL                               ATS DI BERGAMO
  Codice struttura                        Info non disponibile
  Tipo struttura                          Info non disponibile
  Telefono                                Info non disponibile
  Fax                                     Info non disponibile
  Struttura privata                       Info non disponibile
  Prezzo medio                                           22.88
  Name: 337, dtype: objec

## OCR Matching for png

I create a function that taken as input a png, gives me as output a list

In [7]:
reader = easyocr.Reader(['it'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [8]:
# function to match  prescriptions with a dataset from png
def match_recipe_to_dataset_jpg(file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    # Check the input arguments types
    if not isinstance(file, str):
        raise TypeError("The PDF file path must be a string.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("The dataset must be a pandas.DataFrame object.")
    
    # Set-up the reader object and extract the text from the PDF file
    recipe = reader.readtext(file)
    result_string = ""
    for i in range(len(recipe)):
        element = recipe[i][1] # strings are in this position
        result_string += element
        result_string += " "
   
    # Clean the text by removing the dots
    result_string = result_string.replace(".","") 
    # Set-up the regex for matching codes and descriptions
    regex = r"\((\d\w*)\)([A-Z ]+)"
    matches = re.findall(regex,result_string)
    output = []
    # Extract the descriptions from the matches
    for match in matches:
      output.append(match[1])

    # Save the relevant words to match them with the 'Codice prestazione ambulatoriale' column
    words = output 

    # Match the words with the dataset using fuzzywuzzy
    matches = []
    for word in words:
        # Check the input argument type
        if not isinstance(word, str):
            raise TypeError("The word of interest must be a string.")
        # Match the word with the 'Codice prestazione ambulatoriale' column using fuzzywuzzy
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1) # Here we perform fuzzy matching; the 'limit' parameter limits the number of matching results
        # Filter out the results below the threshold
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            # Identify the rows based on the matching code
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows()
            # Append the matches to the output list
            matches.extend([(word, row[1], score[1]) for row in rows])

    # Sort the matches in descending order based on the score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)

    return matches_sorted

In [9]:
match_recipe_to_dataset_jpg('ricetta_2.png', dataset)


[(' ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI ',
  Codice prestazione ambulatoriale    ANTICORPI ANTI CITOPLASMA DEI NEUTROFILI (ANCA)
  Codice branca regionale                                                       90482
  Branca regionale                                     IMMUNOEMATOLOGIA-TRASFUSIONALE
  Tipo di prestazione                                                     LABORATORIO
  Nome struttura                                                  HUMANITAS GAVAZZENI
  Comune struttura                                                            BERGAMO
  Provincia struttura                                                              BG
  Indirizzo                                                    VIA MAURO GAVAZZENI 21
  Descrizione ASL                                                      ATS DI BERGAMO
  Codice struttura                                               Info non disponibile
  Tipo struttura                                                 Info non disponibile
  Tele

## Matching tool

Taking the functions created earlier as a reference, we create a function that takes both pdf and png as input, also modify the output to be a dataframe so that it can be filtered later

In [10]:
# function to match  prescriptions with a dataset and create a new dataframe
def match_recipe_to_dataset_and_create_df(file: str, dataset: pd.DataFrame, threshold: int = 90) -> pd.DataFrame:
    # raise error if file path is not a string
    if not isinstance(file, str):
        raise TypeError("File path must be a string.")
    # raise error if dataset is not a pandas DataFrame
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Dataset must be a pandas.DataFrame.")

    # if file is a pdf, extract text from file
    if file.endswith(".pdf"):
        pdf_file = fitz.open(file) # open pdf file
        text = ''
        for page in pdf_file:
            text += page.get_text() # extract text from each page

        # identify words of interest using regular expression
        words = re.findall(r' - ([A-Za-z\s]+)', text)

    # if file is a png, read text from image using OCR
    elif file.endswith(".png"):
        ricetta = reader.readtext(file) # use tesseract to read text
        result_string = ""
        for i in range(len(ricetta)):
            elemento = ricetta[i][1] #string values are stored in this position in the tuple
            result_string += elemento # concatenate all strings
            result_string += " " # add space between each string
        
        result_string = result_string.replace(".","") # remove dots
        regex = r"\((\d\w*)\)([A-Z ]+)"
        matches = re.findall(regex, result_string) # identify string patterns using regex
        output = []
        for match in matches:
            output.append(match[1]) # append string to output list

        words = output

    # raise error if file format is not supported
    else:
        raise ValueError("File format not supported. Please use a pdf or png file.")

    # match words with values in Codice prestazione ambulatoriale column of dataset using fuzzywuzzy library
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("Word of interest must be a string.")
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1)
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows()
            matches.extend([(word,row[1], score[1]) for row in rows])

    # sort matches by score in descending order
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)
    
    # create empty dictionary to populate with data
    data = {}

    # iterate through list of tuples
    for row in matches_sorted:
        # extract category name and associated row data
        category, row_data, _ = row
        # iterate through columns in row data
        for col, val in row_data.items():
            # create key in dictionary if it doesn't exist already
            if col not in data:
                data[col] = []
            # append value to list of values associated with key
            data[col].append(val)
        # if a column is missing from current row, add a None value
        for col in data.keys():
            if col not in row_data:
                data[col].append(None)

    # create new DataFrame from dictionary
    df = pd.DataFrame(data)

    return df

In [11]:
match_recipe_to_dataset_and_create_df('ricetta_3.png', dataset)

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO,BORGO MANTOVANO,MN,"VIA BUGATTE, 1",ATS DELLA VAL PADANA,30239.0,Struttura di ricovero e cura,0386717111,0386717409,No,3.2
1,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,SERVICE LAB FLEMING RESEARCH,MILANO,MI,VIA BERNARDINO QUARANTA 57,ATS DELLA CITTA' METROPOLITANA DI MILANO,19512.0,Servizio di Medicina di Laboratorio,0292956400,0255230125,Sì,3.2
2,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO CARDIOLOGICO S.P.A.,MILANO,MI,VIA PAREA 4,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
3,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRESIDIO OSPEDALIERO - ASOLA,ASOLA,MN,"PIAZZA 80° FANTERIA, 1",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
4,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSPEDALE LUINO,LUINO,VA,"VIA FORLANINI,6",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMB. IRCCS MONDINO,PAVIA,PV,"VIA MONDINO, 2",ATS DI PAVIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
327,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,IRCCS MULTIMEDICA - MILANO,SESTO SAN GIOVANNI,MI,"VIA MILANESE, 300",ATS DELLA CITTA' METROPOLITANA DI MILANO,30948.0,Struttura di ricovero e cura,02242091,0224209276,Sì,3.2
328,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,P.O. CITTA' DI SESTO S. GIOVANNI,SESTO SAN GIOVANNI,MI,VIALE MATTEOTTI 83,ATS DELLA CITTA' METROPOLITANA DI MILANO,30051.0,Struttura di ricovero e cura,0226251,0226257276,No,3.2
329,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO DIAGNOSI E ANALISI SRL,SUZZARA,MN,"VIA SOMBOR, 16/B",ATS DELLA VAL PADANA,1996.0,Struttura ambulatoriale,0376531661,0376507322,Sì,3.2


In [12]:
output = match_recipe_to_dataset_and_create_df('ricetta_3.pdf', dataset)
output.head()

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
0,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSP.CIVILE DESTRA SECCHIA-PIEVE CORIANO,BORGO MANTOVANO,MN,"VIA BUGATTE, 1",ATS DELLA VAL PADANA,30239.0,Struttura di ricovero e cura,0386717111,0386717409,No,3.2
1,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,SERVICE LAB FLEMING RESEARCH,MILANO,MI,VIA BERNARDINO QUARANTA 57,ATS DELLA CITTA' METROPOLITANA DI MILANO,19512.0,Servizio di Medicina di Laboratorio,0292956400,0255230125,Sì,3.2
2,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO CARDIOLOGICO S.P.A.,MILANO,MI,VIA PAREA 4,ATS DELLA CITTA' METROPOLITANA DI MILANO,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
3,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,PRESIDIO OSPEDALIERO - ASOLA,ASOLA,MN,"PIAZZA 80° FANTERIA, 1",ATS DELLA VAL PADANA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2
4,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO OSPEDALE LUINO,LUINO,VA,"VIA FORLANINI,6",ATS DELL'INSUBRIA,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,Info non disponibile,3.2


In [13]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Codice prestazione ambulatoriale  331 non-null    object
 1   Codice branca regionale           331 non-null    object
 2   Branca regionale                  331 non-null    object
 3   Tipo di prestazione               331 non-null    object
 4   Nome struttura                    331 non-null    object
 5   Comune struttura                  331 non-null    object
 6   Provincia struttura               331 non-null    object
 7   Indirizzo                         331 non-null    object
 8   Descrizione ASL                   331 non-null    object
 9   Codice struttura                  331 non-null    object
 10  Tipo struttura                    331 non-null    object
 11  Telefono                          331 non-null    object
 12  Fax                   

In [14]:
#Function to filter the dataset
#Is possible add more columns to filter, for now we use only 2
def filter_data(df, comune, risposta_strutt):
    filtered_df = df[(df['Comune struttura'] == comune) & 
                     (df['Struttura privata'] == risposta_strutt)]
    return filtered_df

In [15]:
#Try to filter for Comune = Milano and Struttre private = No
filter_data(output,'MILANO','No')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
42,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE S. CARLO BORROMEO - MILANO,MILANO,MI,VIA PIO II 3,ATS DELLA CITTA' METROPOLITANA DI MILANO,30915.0,Struttura di ricovero e cura,0240221,390240222793,No,3.2
47,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO VIA QUARENGHI,MILANO,MI,VIA QUARENGHI N. 21,ATS DELLA CITTA' METROPOLITANA DI MILANO,2456.0,Struttura ambulatoriale,330292742,330292740,No,3.2
52,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,FONDAZ.IRCCS CA' GRANDA - OSPEDALE MAGGIORE PO...,MILANO,MI,"VIA FRANCESCO SFORZA, 28",ATS DELLA CITTA' METROPOLITANA DI MILANO,30925.0,Struttura di ricovero e cura,0255031,0258304350,No,3.2
62,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,P.O. CENTRO TRAUMATOLOGICO ORTOPEDICO - MILANO,MILANO,MI,"VIA BIGNAMI, 1",ATS DELLA CITTA' METROPOLITANA DI MILANO,30912.0,Struttura di ricovero e cura,0257991,0257992808,No,3.2
68,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE CA' GRANDA-NIGUARDA - MILANO,MILANO,MI,PIAZZA OSPEDALE MAGGIORE 3,ATS DELLA CITTA' METROPOLITANA DI MILANO,30913.0,Struttura di ricovero e cura,0264441,026420901,No,3.2
103,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO DON BOSCO,MILANO,MI,"PIAZZA SAN LUIGI, 5 - INGRESSO DA VIA DON BOSC...",ATS DELLA CITTA' METROPOLITANA DI MILANO,3590.0,Struttura ambulatoriale,Info non disponibile,Info non disponibile,No,3.2
122,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO RUGABELLA,MILANO,MI,VIA RUGABELLA 4/6,ATS DELLA CITTA' METROPOLITANA DI MILANO,2461.0,Struttura ambulatoriale,02865986,02865929,No,3.2
124,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,ISTITUTO ORTOPEDICO GAETANO PINI - MILANO,MILANO,MI,"PIAZZA A. CARDINAL FERRARI, 1",ATS DELLA CITTA' METROPOLITANA DI MILANO,30911.0,Struttura di ricovero e cura,02592961,0258300285,No,3.2
137,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,POLIAMBULATORIO VIA LIVIGNO,MILANO,MI,VIA LIVIGNO N. 2/A,ATS DELLA CITTA' METROPOLITANA DI MILANO,2485.0,Struttura ambulatoriale,Info non disponibile,Info non disponibile,No,3.2
142,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,OSPEDALE M. MELLONI - MILANO,MILANO,MI,VIA MACEDONIO MELLONI 52,ATS DELLA CITTA' METROPOLITANA DI MILANO,30057.0,Struttura di ricovero e cura,0263631,0263633241,No,3.2


# Filter dataset with strings

This function does the same thing as the others, but as input it takes a string 

In [16]:
#Function to search structures directly with performance names
def search_words(words):
    matches = []
    data = {}

    for word in words:
        if not isinstance(word, str): #Check if the word is a string, otherwise raise a TypeError
            raise TypeError("The word of interest must be a string.")
        scores = process.extract(word, dataset['Codice prestazione ambulatoriale'], scorer=fuzz.token_set_ratio, limit=1) #Extract matching scores between the word and the dataset
        filtered_scores = [score for score in scores if score[1] >= 70] #Filter scores based on a threshold of 70%
        for score in filtered_scores:
            rows = dataset[dataset['Codice prestazione ambulatoriale'] == score[0]].iterrows() #Get the rows of the dataset that match the word
            matches.extend([(word,row[1], score[1]) for row in rows]) #Extend the matches list with a tuple of the word, the row data, and the matching score
            for row in matches:
                category, row_data, _ = row
                for col, val in row_data.items():
                    if col not in data:
                        data[col] =[]
                    data[col].append(val) #Add the values of therow to the respective column in the data dictionary
                for col in data.keys():
                    if col not in row_data:
                        data[col].append(None) #If a column is missing in the row, add None as a placeholder

    df = pd.DataFrame(data) #Create a pandas DataFrame from the data dictionary
    print(df['Codice prestazione ambulatoriale'].unique()) #Print the unique values of the 'Codice prestazione ambulatoriale' column in the DataFrame
    return df

In [19]:
output_query = search_words(['emocromo','calcitonina'])

['EMOCROMO:  Hb, GR, GB, HCT, PLT, IND. DERIV., F. L.' 'CALCITONINA']


In [20]:
#Try to filter for Bergamo and private strcture
filter_data(output_query,'BERGAMO','Sì')

,Codice prestazione ambulatoriale,Codice branca regionale,Branca regionale,Tipo di prestazione,Nome struttura,Comune struttura,Provincia struttura,Indirizzo,Descrizione ASL,Codice struttura,Tipo struttura,Telefono,Fax,Struttura privata,Prezzo medio
17,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,LABORATORIO BIOANALITICO DELTA SRL,BERGAMO,BG,VIA CLARA MAFFEI 14,ATS DI BERGAMO,1252.0,Servizio di Medicina di Laboratorio,035221565,035235848,Sì,3.2
174,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO BIOMEDICO BERGAMASCO SRL,BERGAMO,BG,ROTONDA DEI MILLE 3/A,ATS DI BERGAMO,1359.0,Servizio di Medicina di Laboratorio,035222332,035236267,Sì,3.2
204,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,HUMANITAS GAVAZZENI,BERGAMO,BG,VIA M. GAVAZZENI 21,ATS DI BERGAMO,30143.0,Struttura di ricovero e cura,0354204111,0354204919,Sì,3.2
348,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,LABORATORIO BIOANALITICO DELTA SRL,BERGAMO,BG,VIA CLARA MAFFEI 14,ATS DI BERGAMO,1252.0,Servizio di Medicina di Laboratorio,035221565,035235848,Sì,3.2
505,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,CENTRO BIOMEDICO BERGAMASCO SRL,BERGAMO,BG,ROTONDA DEI MILLE 3/A,ATS DI BERGAMO,1359.0,Servizio di Medicina di Laboratorio,035222332,035236267,Sì,3.2
535,"EMOCROMO: Hb, GR, GB, HCT, PLT, IND. DERIV., ...",90622,IMMUNOEMATOLOGIA-TRASFUSIONALE,LABORATORIO,HUMANITAS GAVAZZENI,BERGAMO,BG,VIA M. GAVAZZENI 21,ATS DI BERGAMO,30143.0,Struttura di ricovero e cura,0354204111,0354204919,Sì,3.2
679,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,LABORATORIO BIOANALITICO DELTA SRL,BERGAMO,BG,VIA CLARA MAFFEI 14,ATS DI BERGAMO,1252.0,Servizio di Medicina di Laboratorio,035221565,035235848,Sì,14.56
762,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,HUMANITAS GAVAZZENI,BERGAMO,BG,VIA M. GAVAZZENI 21,ATS DI BERGAMO,30143.0,Struttura di ricovero e cura,0354204111,0354204919,Sì,14.56
813,CALCITONINA,90115,CHIMICA CLINICA,LABORATORIO,CENTRO BIOMEDICO BERGAMASCO SRL,BERGAMO,BG,ROTONDA DEI MILLE 3/A,ATS DI BERGAMO,1359.0,Servizio di Medicina di Laboratorio,035222332,035236267,Sì,14.56
